# Machine Learning for building a Classifier

...

## K-Fold Cross-Validation

The full data set is divided into k-subsets.

Each time, one of the l-subsets is used as the test set.
The other k - 1 subsets are put together to be used to train the model.

### Evaluation Metrics

- accuracy = # predicted correctly / # observations
- precision = # predicted as spam that are actually spams / total # predicted as spams.

- recall = # predicted as spam that are actually spams / total # that are actually spams.

## Ensemble Method

Technique that creates multiple models and then combines them to produce better results than any of the single models individually.

### Random Forest

Ensemble learning method that constructs a collection of decision trees and then aggregates the predictions of each tree to determine the final prediction.

__Benefits__
- Can be used for classificagtion or regression
- Easily handles outliers, missing values
- Accepts various types of inputs (continous, ordinal, etc.)
- Less likely to overfit
- Outputs feature importance


In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("../dataset/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']





,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))


In [ ]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

### Explore `RandomForestClassifier` Attributes & Hyperparameters


In [2]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
print(dir(RandomForestClassifier)) 
#feature_importance: powerful tool
#fit: fit the model
#predict: 

print(RandomForestClassifier())
'''
    args: 
        max_depth: 
        n_estimators
'''

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_sample

### Explore `RandomForestClassifier` through Cross-Validation

In [5]:
from sklearn.model_selection import KFold, cross_val_score

In [6]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring = 'accuracy', n_jobs=-1)

array([0.97396768, 0.97755835, 0.97663971, 0.96495957, 0.97214735])

### Explore `RandomForestClassifier` through Holdout Set

In [7]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test =  train_test_split(X_features, data['label'], test_size=0.2)

In [19]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs = -1)

rf_model = rf.fit(X_train, y_train)

In [21]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.0698771325313109, 'body_len'),
 (0.04522519974232611, 7350),
 (0.03714038227397851, 2031),
 (0.03499652929693521, 4796),
 (0.02949174106017828, 1803),
 (0.02013478320011045, 3134),
 (0.01683525270700379, 1361),
 (0.016163935470660046, 7027),
 (0.015762253843493137, 7218),
 (0.014924350831174518, 2299)]

In [22]:
y_pred = rf_model.predict(X_test)

In [26]:
precision, recall, fscore, support = score(y_test, y_pred, pos_label = 'spam', average='binary')
print(f'Precision: {precision:.3f}, Recall {recall:.3f}')

Precision: 1.000, Recall 0.606


__NB__: Recall low means that the model is not aggressive enough to pick up emails as spams

### Build  __`gridsearch`__

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [33]:
def train_RF(n_est, depth):
    RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print(f'Est: {n_est} / Depth {depth} ----- Precision: {precision} / Recall: {recall} / Accuracy: {(y_pred==y_test).sum()/len(y_pred)}')
    

In [34]:
for n_est in [10,50,100]:
    for depth in [10,20,30,None]:
        train_RF(n_est, depth)

Est: 10 / Depth 10 ----- Precision: 1.0 / Recall: 0.6240601503759399 / Accuracy: 0.9551166965888689
Est: 10 / Depth 20 ----- Precision: 1.0 / Recall: 0.5939849624060151 / Accuracy: 0.9515260323159784
Est: 10 / Depth 30 ----- Precision: 1.0 / Recall: 0.6390977443609023 / Accuracy: 0.9569120287253142
Est: 10 / Depth None ----- Precision: 1.0 / Recall: 0.6240601503759399 / Accuracy: 0.9551166965888689
Est: 50 / Depth 10 ----- Precision: 1.0 / Recall: 0.631578947368421 / Accuracy: 0.9560143626570916
Est: 50 / Depth 20 ----- Precision: 1.0 / Recall: 0.631578947368421 / Accuracy: 0.9560143626570916
Est: 50 / Depth 30 ----- Precision: 1.0 / Recall: 0.6466165413533834 / Accuracy: 0.9578096947935368
Est: 50 / Depth None ----- Precision: 1.0 / Recall: 0.6165413533834586 / Accuracy: 0.9542190305206463
Est: 100 / Depth 10 ----- Precision: 1.0 / Recall: 0.5939849624060151 / Accuracy: 0.9515260323159784
Est: 100 / Depth 20 ----- Precision: 1.0 / Recall: 0.6390977443609023 / Accuracy: 0.9569120287253